In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

def resize_image(image, size=(150, 150)):
    return cv2.resize(image, size)

def grayscale_image(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def apply_gaussian_blur(image):
    return cv2.GaussianBlur(image, (5, 5), 0)

def contrast_clahe(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(image)

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = resize_image(image)
    image = grayscale_image(image)
    image = apply_gaussian_blur(image)
    image = contrast_clahe(image)
    return image

def load_data(image_paths, labels):
    images = [preprocess_image(img) for img in image_paths]
    return train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_cnn_model():
    model = models.Sequential()
    
    # CNN layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(5, activation='softmax'))  # 5 stages of Diabetic Retinopathy

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [ ]:
import tensorflow as tf
from preprocess import load_data
from model import build_cnn_model

def train_model(train_images, train_labels, test_images, test_labels):
    model = build_cnn_model()
    
    # Data Augmentation
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.2,
        horizontal_flip=True
    )

    # Fit model
    history = model.fit(datagen.flow(train_images, train_labels, batch_size=32),
                        validation_data=(test_images, test_labels),
                        epochs=20)
    
    return model, history

if __name__ == "__main__":
    # Load and preprocess data
    image_paths = ["data/train_images/{}.png".format(i) for i in range(2000)]  # Example image paths
    labels = [0, 1, 2, 3, 4]  # Corresponding labels for DR stages

    train_images, test_images, train_labels, test_labels = load_data(image_paths, labels)

    # Train the model
    model, history = train_model(train_images, train_labels, test_images, test_labels)
    model.save("diabetic_retinopathy_model.h5")


In [ ]:
import matplotlib.pyplot as plt

def plot_history(history):
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0, 1])
    plt.legend(loc='lower right')
    plt.show()

def load_trained_model(model_path):
    from tensorflow.keras.models import load_model
    return load_model(model_path)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

def build_vgg16_model(input_shape=(150, 150, 3)):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model layers
    
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))  # Regularization
    model.add(layers.Dense(5, activation='softmax'))  # 5 classes for DR stages
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
import tensorflow as tf
from preprocess import load_data
from vgg_model import build_vgg16_model

def train_vgg_model(train_images, train_labels, test_images, test_labels):
    model = build_vgg16_model()
    
    # Data Augmentation
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=30,
        zoom_range=0.2,
        horizontal_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1
    )
    
    # Early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    
    # Fit model
    history = model.fit(datagen.flow(train_images, train_labels, batch_size=32),
                        validation_data=(test_images, test_labels),
                        epochs=30,
                        callbacks=[early_stopping])
    
    return model, history

if __name__ == "__main__":
    # Load and preprocess data
    image_paths = ["data/train_images/{}.png".format(i) for i in range(2000)]  # Placeholder paths
    labels = [0, 1, 2, 3, 4]  # Labels for DR stages
    
    train_images, test_images, train_labels, test_labels = load_data(image_paths, labels)
    
    # Train the VGG16 model
    model, history = train_vgg_model(train_images, train_labels, test_images, test_labels)
    model.save("diabetic_retinopathy_vgg_model.h5")


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import numpy as np
from utils import load_trained_model
from preprocess import preprocess_image

def evaluate_model(test_images, test_labels, model_path):
    model = load_trained_model(model_path)
    predictions = np.argmax(model.predict(test_images), axis=1)
    
    # Confusion Matrix
    cm = confusion_matrix(test_labels, predictions)
    sns.heatmap(cm, annot=True, fmt='d')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()
    
    # Classification Report
    print(classification_report(test_labels, predictions, target_names=['Normal', 'Mild', 'Moderate', 'Severe', 'Proliferative']))

if __name__ == "__main__":
    # Example of evaluation
    test_image_paths = ["data/test_images/{}.png".format(i) for i in range(500)]  # Placeholder paths
    test_images = [preprocess_image(img) for img in test_image_paths]
    test_labels = [0, 1, 2, 3, 4]  # Replace with actual labels

    evaluate_model(test_images, test_labels, "diabetic_retinopathy_vgg_model.h5")


In [ ]:
import tensorflow as tf

def load_trained_model(model_path):
    return tf.keras.models.load_model(model_path)

def plot_accuracy_loss(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Test'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Test'], loc='upper left')

    plt.show()


In [ ]:
import cv2

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = resize_image(image)
    image = apply_clahe(image)  # Apply CLAHE
    image = normalize_image(image)  # Normalize intensity
    return image

def apply_clahe(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(gray)

def normalize_image(image):
    return cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
